In [1]:
%load_ext autoreload
%autoreload 2

import os
import pathlib
cwd = pathlib.Path.cwd()
if cwd.name != 'ryn':
    print('changing directory')
    os.chdir(cwd.parent)

import logging
logger = logging.getLogger()

# seeds = [30061990, 8051991, 25031990, 2041992]
seeds = [30061990, ]
logger.setLevel(logging.INFO)

changing directory


In [2]:
from ryn.graphs import split
from ryn.graphs import loader

g = loader.load_graphs_from_uri('oke.fb15k237-trainvalidtest')[0]
print(f'loaded {g.str_stats}')

rels = split.Relation.from_graph(g)
rels.sort(key=lambda rel: rel.ratio)
print(f'retrieved {len(rels)} relations')

loaded ryn graph: oke.fb15k237-trainvalidtest
  nodes: 14541
  edges: 310116 (237 types)
  degree:
    mean 42.65
    median 26



retrieved 237 relations


In [ ]:
from tabulate import tabulate

rows = [(r.r, r.ratio, len(r.hs), len(r.ts), r.name) for r in rels]

N = 5

print(f'first {N}')
print(tabulate(rows[:N]))

print(f'mid {N}')
m = len(rows) / 2
print(tabulate(rows[int(m-N/2):int(m+N/2)]))

print(f'last {N}')
print(tabulate(rows[-N:]))

In [ ]:
import ryn
from ryn.common import plotter

path = ryn.ENV.SPLIT_DIR / f'{g.name}.ratio'
path.parent.mkdir(exist_ok=True, parents=True)

plt = plotter.Plotter(title=f'Ratio Distribution {g.name}', xlabel='Relation', ylabel='Ratio', fname=str(path))
plt.ax.scatter(range(len(rels)), [r.ratio for r in rels], color=plotter.CLR[0], s=1)

print()
plt.plot()
print()

In [79]:
import ryn

for i in range(5, 6):

    cfg = split.Config(
        ow_split=0.5,
        train_split=0.7,
        threshold=i * 10)

    print(cfg)

    for _ in split.create(g, cfg, seeds):
        print('done')


Config:
  ow split: 0.5
  train split: 0.7
  relation threshold: 50
  git: 4c3bfc0dead1ef01cf1cfa873d532c15d0cabfff



done


In [83]:
path = pathlib.Path('data') / 'split' / 'oke.fb15k237_0.50-0.70_50_30061990'
# split.analyse(path)
dataset = split.Dataset.load(path)
print(dataset)

RYN.SPLIT DATASET
-----------------

6054 retained concepts

Config:
  ow split: 0.5
  train split: 0.7
  relation threshold: 50
  git: 4c3bfc0dead1ef01cf1cfa873d532c15d0cabfff

ryn graph: oke.fb15k237-trainvalidtest
  nodes: 14541
  edges: 310116 (237 types)
  degree:
    mean 42.65
    median 26

Closed World - TRAIN:
  owe: 9568
  triples: 113038
  entities: 9568
  heads: 8756
  tails: 8507
Closed World - VALID:
  owe: 15
  triples: 41910
  entities: 4044
  heads: 3649
  tails: 3520
Open World - VALID:
  owe: 4658
  triples: 108541
  entities: 12516
  heads: 10798
  tails: 10749
Open World - TEST:
  owe: 300
  triples: 46627
  entities: 8469
  heads: 6946
  tails: 6836
